### 한국어 위키피디아로 Word2Vec 학습

https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

In [249]:
import os
import re
from tqdm import tqdm
from konlpy.tag import Okt
import xml.etree.ElementTree as ET
from gensim.models import Word2Vec

In [250]:
file_path = 'kowiki-latest-pages-articles.xml'

docs = []

### 데이터 전처리

In [251]:
def clean_text(text):
    #text = re.sub(r'\[[^\]]*\]', '', text)
    text = text.replace('{{위키데이터 속성 추적}}', '')
    
    text = re.sub(r'\'', '', text)
    text = re.sub(r'\{\{', '', text)
    text = re.sub(r'\}\}', '', text)
    text = re.sub(r'\[\[', '', text)
    text = re.sub(r'\]\]', '', text)
    
    text = re.sub(r'<[^>]*>', '', text)
    text = re.sub(r'\|', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

In [252]:
test_text = "'{{위키데이터 속성 추적}} [[파일:Aozora Bunko Logo.png|섬네일|아오조라 문고 로고]] 아오조라 문고({{llang|ja|{{ruby-ja|靑空文庫|あおぞらぶんこ}}|아오조라 분고}}{{해석|푸른하늘 문고}})는 ‘[[일본어]]판"
print(clean_text(test_text))

파일:Aozora Bunko Logo.png 섬네일 아오조라 문고 로고 아오조라 문고(llang ja ruby-ja 靑空文庫 あおぞらぶんこ 아오조라 분고해석 푸른하늘 문고)는 ‘일본어판


In [253]:
def parse_wiki_dump(file_path, maxcount=0):
    context = ET.iterparse(file_path, events=('start', 'end'))
    context = iter(context)
    event, root = next(context)

    i = 0

    for event, elem in tqdm(context):

        if maxcount != 0 and i > maxcount:
            break

        if event == 'end' and elem.tag == '{http://www.mediawiki.org/xml/export-0.10/}page':
            title_elem = elem.find('{http://www.mediawiki.org/xml/export-0.10/}title')
            ns_elem = elem.find('{http://www.mediawiki.org/xml/export-0.10/}ns')
            id_elem = elem.find('{http://www.mediawiki.org/xml/export-0.10/}id')
            revision_elem = elem.find('{http://www.mediawiki.org/xml/export-0.10/}revision')
            text_elem = revision_elem.find('{http://www.mediawiki.org/xml/export-0.10/}text') if revision_elem is not None else None

            title = title_elem.text if title_elem is not None else ''
            ns = ns_elem.text if ns_elem is not None else ''
            id_ = id_elem.text if id_elem is not None else ''
            text = text_elem.text if text_elem is not None else ''

            if text:
                cleaned_text = clean_text(text)
                docs.append(cleaned_text)
                #print(f'Title: {title}')
                #print(f'Namespace: {ns}')
                #print(f'ID: {id_}')
                #print(f'Cleaned Text: {cleaned_text[:200]}...')  # Print first 200 characters of cleaned text
                #print('='*80)

            root.clear()

        i += 1
    return

In [254]:
parse_wiki_dump(file_path, 1000000)

1000001it [00:20, 49953.75it/s]


In [255]:
docs[:15]

['위키백과:소개 위키백과 우리 모두가 만들어가는 자유 백과사전문서 위키백과:통계 말풍선 1=NUMBEROFARTICLES style=text-decoration: none; cursor: pointer; 2=넘겨주기와 막다른 문서를 제외한 모든 글의 수개와 최근 기여자 위키백과:통계 말풍선 1=NUMBEROFACTIVEUSERS style=text-decoration: none; cursor: pointer; 2=비회원 사용자를 제외한 최근 기여자 수명 파일: XEIcon xi-group.svg 40px sub link=위키백과:사랑방 위키백과:사랑방 사랑방 다른 분들과 의견을 교환해봐요! 파일: XEIcon xi-message.svg 40px sub link=위키백과:질문방 위키백과: 질문방 질문방 지침으로 해소되지 않는 의문을 풀어봐요! 파일:Group half.svg 40px sub link=위키백과:사용자 모임 위키백과:사용자 모임 사용자 모임 커뮤니티 포털에 방문해 보세요! 파일: XEIcon xi-library-bookmark.svg 40px sub link=위키백과:정책과 지침 위키백과:정책과 지침 정책과 지침 편집하시기 전에 한번 읽어주세요! 위키백과:위키백과에 참여하기 위키백과 참여하기 File:XEIcon xi-heart.svg 40px sub link=위키백과:환영합니다 위키백과:환영합니다 환영합니다! 위키백과는 여러분을 환영합니다.이것부터 시작해 볼까요? File:Book_sans_information.svg 40px sub link=도움말:소개 도움말:소개 위키백과 길라잡이 여러분을 위키인으로 만들어줄 가이드북! File:Document sans add.svg 40px sub link=도움말:새 문서 만들기 도움말:새 문서 만들기 새 문서 만들기 도움말 새 문서를 만드는 방법에 대해 안내합니다. File:Document text edit.svg 40px sub link=도움말:문서 편집하기 도움말:문서 편집하기 문서 편집 도움말 문서를

In [256]:
len(docs)

26995

### 토큰화

In [257]:
stopwords=['의','가','이','은','들','는','좀','잘','과','도','를','으로','자','에','와','한','하다']

result = []

def morpheme_analysis():
    okt = Okt()

    for doc in tqdm(docs):

        if len(doc) < 30:
            continue

        temp = okt.morphs(doc)
        temp = [word for word in temp if not word in stopwords]
        result.append(temp)

    return

In [258]:
morpheme_analysis()

100%|██████████| 26995/26995 [18:41<00:00, 24.08it/s] 


In [259]:
result[:1]

[['위키',
  '백',
  ':',
  '소개',
  '위키',
  '백',
  '우리',
  '모두',
  '만들어가는',
  '자유',
  '백과사전',
  '문서',
  '위키',
  '백',
  ':',
  '통계',
  '말풍선',
  '1',
  '=',
  'NUMBEROFARTICLES',
  'style',
  '=',
  'text',
  '-',
  'decoration',
  ':',
  'none',
  ';',
  'cursor',
  ':',
  'pointer',
  ';',
  '2',
  '=',
  '넘겨주기',
  '막',
  '다른',
  '문서',
  '제외',
  '모든',
  '글',
  '수',
  '개',
  '최근',
  '기',
  '여자',
  '위키',
  '백',
  ':',
  '통계',
  '말풍선',
  '1',
  '=',
  'NUMBEROFACTIVEUSERS',
  'style',
  '=',
  'text',
  '-',
  'decoration',
  ':',
  'none',
  ';',
  'cursor',
  ':',
  'pointer',
  ';',
  '2',
  '=',
  '비회',
  '원',
  '사용자',
  '제외',
  '최근',
  '기',
  '여자',
  '수명',
  '파일',
  ':',
  'XEIcon',
  'xi',
  '-',
  'group',
  '.',
  'svg',
  '40',
  'px',
  'sub',
  'link',
  '=',
  '위키',
  '백',
  ':',
  '사랑방',
  '위키',
  '백',
  ':',
  '사랑방',
  '사랑방',
  '다른',
  '분',
  '의견',
  '을',
  '교환',
  '해봐요',
  '!',
  '파일',
  ':',
  'XEIcon',
  'xi',
  '-',
  'message',
  '.',
  'svg',
  '40',
  'px',
  'sub',
  'li

In [260]:
len(result)

22188

### Word2Vec 학습

In [261]:
model = Word2Vec(sentences = result, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [262]:
model.wv.save_word2vec_format('kor_wiki_w2v')

In [263]:
model_result = model.wv.most_similar("대한민국")
print(model_result)

[('조선민주주의인민공화국', 0.5955079197883606), ('한국', 0.5861400961875916), ('중화민국', 0.573898434638977), ('일본', 0.5564777255058289), ('미국', 0.525367796421051), ('1988년', 0.5179989337921143), ('일제강점기', 0.5126249194145203), ('중화인민공화국', 0.5125312209129333), ('장관', 0.5112230181694031), ('서울', 0.49893438816070557)]


In [264]:
model_result = model.wv.most_similar("과학")
print(model_result)

[('천문학', 0.7143200039863586), ('심리학', 0.7077535390853882), ('사회과학', 0.7050596475601196), ('기술', 0.6878145933151245), ('물리학', 0.6864867806434631), ('철학', 0.6861381530761719), ('과학철학', 0.6834330558776855), ('인류학', 0.6786378622055054), ('사회학', 0.6768509745597839), ('인문학', 0.661012589931488)]


In [265]:
model_result = model.wv.most_similar("소금")
print(model_result)

[('기름', 0.8482269644737244), ('채소', 0.8393579125404358), ('포도주', 0.8392421007156372), ('달걀', 0.8275828957557678), ('감자', 0.8270576000213623), ('흙', 0.8269113302230835), ('가축', 0.818506121635437), ('포도', 0.8172891139984131), ('생선', 0.813700258731842), ('과일', 0.8133699297904968)]
